In [1]:

!pip install -q transformers datasets accelerate peft evaluate seqeval bitsandbytes
# !pip install -U bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 32.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 39.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.1 MB/s eta 0:00:00:00:0100:0

## DataPrep

In [7]:

import pandas as pd
import json

# ---- CONFIG ----
input_file = "/kaggle/input/datset-csv/train_dataset.csv"   # Your dataset file
output_file = "/kaggle/working/qlora_dataset.jsonl"

# Define system prompts for each task type
# system_prompts = {
#     "re_ko": "You are an expert IT assistant responsible for generating high-quality Knowledge Articles. The user has modified certain fields from your previous response to better reflect their specific issue. Using the updated information, regenerate the Knowledge Article with enhanced accuracy and depth. Ensure that each section—short description, detailed long description with context, root causes, observable symptoms, and resolution note—is comprehensive, technically precise, and aligned with industry best practices. The resolution note must be detailed (150+ words), step-by-step, and actionable, covering all relevant scenarios with practical commands or procedures.",
#     "ko" : "You are an expert IT assistant for Knowledge Article generation. Analyze the user query to identify the core technical issue and generate comprehensive documentation. Provide accurate short description, detailed long description with context, root causes (underlying technical factors), observable symptoms that indicate the problem, and most importantly a step-by-step 150+ word detailed Resolution_note that covers all scenarios with practical commands/solutions. Ensure each field is technically accurate, and should not contain any senstive details, professionally written, and provides actionable guidance for IT professionals. You must provide only one complete object of 'KO' class where each field is thoroughly detailed and validated",
#     "causecode": "You are an ITSM assistant focused on incident diagnostics. Generate nine two to three word causes for the given incident, ensuring each cause is concise and clear, where relevancy should be between 0-100.",
#     "5_why": "You are an advanced Information technology service management assistant that provides five why root cause analysis, I want five why which covers all the cases from the given problem statement without any additional explanation for the question. Consider all the major aspects of given problem statement, five root cause analysis should contain only five questions and answers.",
#     "fishbone": "You are an expert ITSM assistant specializing in identifying underlying causes of issues. Using the Fishbone (Ishikawa) method, analyze the following user query and categorize potential root causes across key dimensions. Present the findings in a structured format.",
#     "sip": "You are an ITSM assistant specializing in Service Improvement Plans (SIP). Your task is to provide concise and actionable SIP proposals for the given problem statement, ensuring that all key aspects of the issue are addressed comprehensively",
#     "ps": "You are an advanced Information technology service management assistant that provides problem statement to the given incident. Consider all the major aspects of given incident_text; this is crucial information and and problem statement should contain maximum ten words.and mark it as Q1,Q2,Q3... seprated by newline",
#     "re_causecode":"You are a highly skilled IT assistant specialized in root cause analysis, tasked with examining the provided scenario to identify core issues, enhance existing causes, and generate nine new precise causes (maximum 3 words each), prioritized by severity.",
#     "re_ps":"You are an expert IT assistant focused on comprehensive problem statement analysis, responsible for transforming the given scenario into clear, structured problem statements (labeled Q1, Q2, Q3...), incorporating system impact metrics, business criticality indicators, time sensitivity factors, and interdependencies while ensuring each statement follows SMART criteria (Specific, Measurable, Achievable, Relevant, Time-bound) for actionable issue resolution, mark questions with Q1,Q2,Q3... seprated by newline",
#     "e_5_why":"You are an experienced IT assistant proficient in the Five Whys methodology, tasked with conducting a thorough root cause investigation by generating strategic follow-up questions and detailed answers that incorporate technical context, system dependencies, impact assessment, stakeholder considerations, and industry standards, culminating in actionable insights and preventive measures for long-term resolution.",
#     "re_sip":"You are a skilled IT assistant specializing in Service Improvement Plans (SIP), focused on enhancing provided improvement strategies by incorporating ITIL best practices, measurable KPIs, clear implementation timelines, resource requirements, risk assessments, stakeholder responsibilities, and success criteria while ensuring alignment with business objectives and technical feasibility.",
#     "re_fishbone":"You are an expert IT assistant in Fishbone (Ishikawa) analysis, dedicated to developing comprehensive cause-and-effect diagrams that examine six key categories (Methods, Machines, Materials, Measurements, Environment, People), incorporating detailed technical factors, systemic relationships, data-driven insights, and industry benchmarks to identify root causes and propose effective remediation strategies."
# }

system_prompts = {
    # "re_ko": "You are an expert IT assistant responsible for generating high-quality Knowledge Articles. The user has modified certain fields from your previous response to better reflect their specific issue. Using the updated information, regenerate the Knowledge Article with enhanced accuracy and depth. Ensure that each section—short description, detailed long description with context, root causes, observable symptoms, and resolution note—is comprehensive, technically precise, and aligned with industry best practices. The resolution note must be detailed (150+ words), step-by-step, and actionable, covering all relevant scenarios with practical commands or procedures.",
    # "ko" : "You are an expert IT assistant for Knowledge Article generation. Analyze the user query to identify the core technical issue and generate comprehensive documentation. Provide accurate short description, detailed long description with context, root causes (underlying technical factors), observable symptoms that indicate the problem, and most importantly a step-by-step 150+ word detailed Resolution_note that covers all scenarios with practical commands/solutions. Ensure each field is technically accurate, and should not contain any senstive details, professionally written, and provides actionable guidance for IT professionals. You must provide only one complete object of 'KO' class where each field is thoroughly detailed and validated",
    # "ko":'You are an expert IT assistant for Knowledge Article generation. Analyze the user query to identify the core technical issue and generate comprehensive documentation. Provide accurate short description, detailed long description with context, root causes (underlying technical factors), observable symptoms that indicate the problem, and most importantly a step-by-step 150+ word detailed Resolution_note that covers all scenarios with practical commands/solutions. Ensure each field is technically accurate, and should not contain any senstive details, professionally written, and provides actionable guidance for IT professionals. You must provide only one complete object of \'KO\' class where each field is thoroughly detailed and validatedFormat your response as follows: The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"Query": {"descrption": "the root cause of the problem", "title": "Query", "type": "string"}, "Symptoms": {"description": "Symptoms refer to the observable signs, behaviors, or effects that indicate the presence of a problem/issue", "title": "Symptoms", "type": "string"}, "Short_description": {"description": "A brief and short description of user query", "title": "Short Description", "type": "string"}, "Long_description": {"description": "A detailed and good enhanced description of user query", "title": "Long Description", "type": "string"}, "Causes": {"description": "Causes refer to the underlying reasons or factors that lead to the occurrence of a problem or issue", "title": "Causes", "type": "string"}, "Resolution_note": {"description": "step by step detailed Enhanced kowledge article which covers all the scenerio", "title": "Resolution Note", "type": "string"}}, "required": ["Query", "Symptoms", "Short_description", "Long_description", "Causes", "Resolution_note"]}\n```',
    "ko": 'You are an expert IT assistant for Knowledge Article generation. Analyze the query and produce one complete KO object with detailed symptoms, descriptions, causes, and a 150+ word actionable Resolution_note. Output must strictly follow the JSON structure below exactly as written.\n```json\n{\n  \"Query\": \"...\",\n  \"Symptoms\": \"...\",\n  \"Short_description\": \"...\",\n  \"Long_description\": \"...\",\n  \"Causes\": \"...\",\n  \"Resolution_note\": \"..\"\n}\n```',
    # "causecode": "You are an ITSM assistant. Generate nine concise causes (2–3 words each) for the incident, with relevancy scores (0–100).",

    "5_why": "You are an ITSM assistant. Perform a 5 Whys root cause analysis for the given issue. Provide exactly five question-answer pairs without extra explanation.",

    "fishbone": "You are an ITSM assistant. Perform a Fishbone analysis for the given issue. Categorize root causes under key dimensions in a structured format.",

    "sip": "You are an ITSM assistant. Provide a concise, actionable Service Improvement Plan (SIP) for the given issue, covering all key aspects.",

    "ps": "You are an ITSM assistant. Generate problem statements (max 10 words each) for the given incident. Label them Q1, Q2, Q3... separated by newlines.",

    "re_causecode": "You are an IT assistant. Enhance existing causes and generate nine new precise causes (max 3 words each), prioritized by severity.",

    "re_ps": "You are an IT assistant. Transform the scenario into SMART problem statements (Specific, Measurable, Achievable, Relevant, Time-bound). Label Q1, Q2, Q3... separated by newlines.",

    "re_5_why": "You are an IT assistant. Perform an advanced 5 Whys analysis with technical context, dependencies, impact, and preventive measures for long-term resolution.",

    "re_sip": "You are an IT assistant. Enhance the SIP using ITIL best practices, KPIs, timelines, resources, risks, and success criteria aligned with business objectives.",

    "re_fishbone": "You are an IT assistant. Create a detailed Fishbone diagram covering six categories (Methods, Machines, Materials, Measurements, Environment, People) with technical factors and remediation strategies."
}

df = pd.read_csv(input_file)

print(df['type'].unique())
print(df[df['type'].isin(["ko","re_ko"])].shape)


['re_ps:7' 're_ps:5' 're_sip' '5_why' 'ps:5' 're_fishbone' 're_ko' 'ps:8'
 'causecode' 're_causecode' 'ko' 'sip' 're_5_why' 'fishbone' 're_ps:8'
 'ps:7' 'ps:4' 're_ps:6' 'ps:6' 're_ps:4']
(1902, 3)


In [3]:


with open(output_file, "w", encoding="utf-8") as f_out:
    df = pd.read_csv(input_file)
    for _, row in df.iterrows():
        query = str(row["query"]).strip()
        target = str(row["target"]).strip()
        type_field = str(row["type"]).strip()

        if type_field in ['ko','re_ko']:
            parts = type_field.split(":")
            task_type = parts[0]  # e.g., ps or re_ps
            count = parts[1] if len(parts) > 1 else None
    
            # Build instruction dynamically
            instruction = f"{query}\n"
            if count:
                instruction += f"\nGenerate {count} problem statements."
    
            system_prompt = system_prompts.get(task_type, "You are an IT assistant. Provide the best possible solution.")
    
            record = {
                "system": system_prompt,
                "instruction": instruction,
                "response": target
            }
    
            f_out.write(json.dumps(record) + "\n")

print(f"✅ Dataset saved to {output_file}")


✅ Dataset saved to /kaggle/working/qlora_dataset.jsonl


In [91]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# model_name = "Qwen/Qwen2.5-3B-Instruct"
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,  # QLoRA magic
    # device_map="balanced",
    device_map = None,
    torch_dtype=torch.float16
)

model.save_pretrained("base_model")
tokenizer.save_pretrained("base_model")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


('base_model/tokenizer_config.json',
 'base_model/special_tokens_map.json',
 'base_model/chat_template.jinja',
 'base_model/vocab.json',
 'base_model/merges.txt',
 'base_model/added_tokens.json',
 'base_model/tokenizer.json')

In [73]:
print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [5]:

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # Common for attention layers
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)




trainable params: 18,464,768 || all params: 1,562,179,072 || trainable%: 1.1820
Using device: cuda


In [6]:

from datasets import load_dataset

dataset = load_dataset("json", data_files=output_file)
dataset = dataset["train"]
dataset = dataset.shuffle(seed=42)


# split_dataset = dataset.train_test_split(test_size=0.1)
# train_dataset = split_dataset["train"]
# eval_dataset = split_dataset["test"]



train_dataset = dataset
dataset


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['system', 'instruction', 'response'],
    num_rows: 1902
})

In [7]:
# eval_dataset, train_dataset

In [8]:
import json
import ast

def convert_issue_string(input_str, query="", aet=30, resolution_summary=""):

    data = ast.literal_eval(input_str)

    # Build final structure
    output = {
        # "Query": query,
        "Symptoms": data.get("symptoms", None),
        "Short_description": data.get("short_description", ""),
        "Long_description": data.get("long_description", ""),
        "Causes": data.get("causes", ""),
        "Resolution_note": data.get("resolution_note", "")
    }

    json_output = json.dumps(output, indent=2)
    return f"```json\n{json_output}\n```", data.get("symptoms", None),

In [9]:
from typing import List, Dict, Any

# choose a sensible max_length (<= model context length)
DEFAULT_MAX_LENGTH = min(getattr(tokenizer, "model_max_length", 4096), 4096)
def tokenize_batch(batch: Dict[str, List[str]],
                   max_length: int = DEFAULT_MAX_LENGTH,
                   drop_truncated: bool = True) -> Dict[str, List]:

    instructions = batch["instruction"]
    responses = batch["response"]
    N = len(instructions)

    input_ids_list = []
    attn_list = []
    labels_list = []
    keep_mask = []
    token_stats = []   # <-- collect only for kept examples

    for i in range(N):
        system = system_prompts.get("ko") or ""
        instruction = instructions[i] or ""
        response, cont = convert_issue_string(responses[i]) or ""

        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": instruction},
            {"role": "assistant", "content": response},
        ]

        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)

        prompt_msgs = [
            {"role": "system", "content": system},
            {"role": "user", "content": instruction},
            {"role": "assistant", "content": ""},
        ]
        prompt_text = tokenizer.apply_chat_template(prompt_msgs, tokenize=False, add_generation_prompt=False)

        enc_prompt = tokenizer(prompt_text, truncation=False, padding=False)
        prompt_len = len(enc_prompt["input_ids"])

        enc_full = tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_attention_mask=True
        )

        input_ids = enc_full["input_ids"]
        attention_mask = enc_full["attention_mask"]

        if prompt_len >= max_length:
            if drop_truncated:
                # mark as dropped — do NOT append token_stats for this example
                keep_mask.append(False)
                input_ids_list.append([0] * max_length)   # placeholders
                attn_list.append([0] * max_length)
                labels_list.append([-100] * max_length)
                continue
            else:
                # keep it but labels are all masked
                labels = [-100] * max_length
        else:
            labels = input_ids.copy()
            # mask prompt token positions
            for idx in range(min(prompt_len, max_length)):
                labels[idx] = -100
            # mask padding tokens
            for idx, m in enumerate(attention_mask):
                if m == 0:
                    labels[idx] = -100

        # ====== compute token stats for a kept example ======
        trainable_tokens = sum(1 for x in labels if x != -100)
        total_tokens = len(input_ids)
        prompt_tokens = prompt_len

        token_stats.append({
            "total_tokens": total_tokens,
            "prompt_tokens": prompt_tokens,
            "trainable_tokens": trainable_tokens
        })
        # =====================================================

        keep_mask.append(True)
        input_ids_list.append(input_ids)
        attn_list.append(attention_mask)
        labels_list.append(labels)

    # Filter out dropped examples from other lists (token_stats already matches kept list length)
    if any(k is False for k in keep_mask):
        input_ids_list = [x for x, k in zip(input_ids_list, keep_mask) if k]
        attn_list = [x for x, k in zip(attn_list, keep_mask) if k]
        labels_list = [x for x, k in zip(labels_list, keep_mask) if k]
        # token_stats was only created for kept examples, so no filtering required

    # Final sanity check (helps catch mismatches early)
    L = len(input_ids_list)
    assert L == len(attn_list) == len(labels_list) == len(token_stats), (
        f"Length mismatch after filtering: input_ids {len(input_ids_list)}, "
        f"attn {len(attn_list)}, labels {len(labels_list)}, token_stats {len(token_stats)}"
    )

    return {
        "input_ids": input_ids_list,
        "attention_mask": attn_list,
        "labels": labels_list,
        "token_stats": token_stats
    }



In [10]:
# preferred: use batched=True
tokenized_train = train_dataset.map(
    lambda batch: tokenize_batch(batch, max_length=1024, drop_truncated=True),
    batched=True,
    batch_size=1000,        # tune per memory (try 256/512/1000)
    remove_columns=train_dataset.column_names,
    num_proc=1             
)

# tokenized_eval = eval_dataset.map(
#     lambda batch: tokenize_batch(batch, max_length=1024, drop_truncated=False),
#     batched=True,
#     batch_size=1000,
#     remove_columns=eval_dataset.column_names
# )


Map (num_proc=1):   0%|          | 0/1902 [00:00<?, ? examples/s]

In [11]:
stats = tokenized_train["token_stats"]

total_trainable = sum(s["trainable_tokens"] for s in stats)
print("Total trainable tokens:", total_trainable)


Total trainable tokens: 552347


In [13]:
# tokenized_train, tokenized_eval, system_prompts.get("ko")

In [14]:

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./qwen-qlora-it-2-v6",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    num_train_epochs=10,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=200,
    report_to="none",
    lr_scheduler_type="cosine",   # or "linear", "polynomial", "cosine_with_restarts"
    warmup_steps=25    
)



In [15]:
# =========================== langchian ===============================
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnableLambda

from pydantic import BaseModel, Field
class KO(BaseModel):
    Symptoms:str=Field(...,description='Symptoms refer to the observable signs, behaviors, or effects that indicate the presence of a problem/issue')
    Short_description:str=Field(...,description='A brief and short description of user query')
    Long_description:str=Field(...,description='A detailed and good enhanced description of user query')
    Causes:str=Field(...,description='Causes refer to the underlying reasons or factors that lead to the occurrence of a problem or issue')
    Resolution_note:str=Field(...,description='step by step detailed Enhanced kowledge article which covers all the scenerio')
    
parser = PydanticOutputParser(pydantic_object=KO)

In [16]:
from transformers import Trainer
import torch
import torch.nn as nn
import torch.nn.functional as F

class StableTrainer(Trainer):
    def __init__(self, *args, tokenizer=tokenizer, output_parser=parser, json_penalty: float = 0.7, **kwargs):
        super().__init__(*args, **kwargs)
        if tokenizer is None:
            raise ValueError("tokenizer must be provided")
        self.tokenizer = tokenizer
        self.output_parser = output_parser
        self.json_penalty = float(json_penalty)
        
        # Cache special token IDs for efficiency
        self.brace_open_id = tokenizer.encode("{", add_special_tokens=False)[0]
        self.brace_close_id = tokenizer.encode("}", add_special_tokens=False)[0]
        self.bracket_open_id = tokenizer.encode("[", add_special_tokens=False)[0]
        self.bracket_close_id = tokenizer.encode("]", add_special_tokens=False)[0]
        
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        Compute loss with optional differentiable JSON structure penalty.
        Base loss computation matches Trainer exactly.
        """
        # Forward pass
        outputs = model(**inputs)
        
        # ========== BASE LOSS (EXACT MATCH TO TRAINER) ==========
        # If model returns loss, use it directly
        if hasattr(outputs, "loss") and outputs.loss is not None:
            base_loss = outputs.loss
        else:
            # Compute cross-entropy loss manually
            logits = outputs.logits
            labels = inputs.get("labels")
            
            if labels is None:
                raise ValueError("No labels provided and model didn't return loss")
            
            # Shift for causal LM: predict next token
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            
            # Flatten and compute loss
            loss_fct = nn.CrossEntropyLoss()
            base_loss = loss_fct(
                shift_logits.view(-1, shift_logits.size(-1)),
                shift_labels.view(-1)
            )
        
        # ========== JSON STRUCTURE PENALTY (DIFFERENTIABLE) ==========
        json_loss = torch.tensor(0.0, device=base_loss.device)
        
        if self.json_penalty > 0.0 and self.output_parser is not None:
            logits = outputs.logits
            
            # Get probabilities for structural tokens (differentiable)
            probs = F.softmax(logits, dim=-1)  # [batch, seq_len, vocab]
            
            # Extract probabilities for key structural tokens
            brace_open_prob = probs[:, :, self.brace_open_id]   # [batch, seq_len]
            brace_close_prob = probs[:, :, self.brace_close_id]
            bracket_open_prob = probs[:, :, self.bracket_open_id]
            bracket_close_prob = probs[:, :, self.bracket_close_id]
            
            # Penalty 1: Encourage balanced braces/brackets (soft constraint)
            # Sum probabilities across sequence
            total_brace_open = brace_open_prob.sum(dim=1)
            total_brace_close = brace_close_prob.sum(dim=1)
            total_bracket_open = bracket_open_prob.sum(dim=1)
            total_bracket_close = bracket_close_prob.sum(dim=1)
            
            # Penalize imbalance
            brace_imbalance = F.relu(torch.abs(total_brace_open - total_brace_close) - 0.5)
            bracket_imbalance = F.relu(torch.abs(total_bracket_open - total_bracket_close) - 0.5)
            
            balance_penalty = (brace_imbalance + bracket_imbalance).mean()
            
            # Penalty 2: Encourage starting with '{' or '[' 
            # Look at first few tokens
            start_probs = probs[:, :5, :]  # [batch, 5, vocab]
            start_struct_prob = (
                start_probs[:, :, self.brace_open_id] + 
                start_probs[:, :, self.bracket_open_id]
            ).max(dim=1)[0]  # [batch]
            
            start_penalty = (1.0 - start_struct_prob).mean()
            
            # Combine penalties
            json_loss = self.json_penalty * (balance_penalty + 0.5 * start_penalty)
        
        # ========== TOTAL LOSS ==========
        total_loss = base_loss + json_loss
        
        return (total_loss, outputs) if return_outputs else total_loss

In [17]:

from transformers import Trainer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = StableTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Trainer.tokenizer is now deprecated. You should use `Trainer.processing_class = processing_class` instead.


In [18]:

trainer.train()

Step,Training Loss
10,18.738100
20,15.162600
30,10.543600
40,7.858100
50,6.986400
60,6.593400
70,6.291500
80,6.108700
90,6.059800
100,5.872300


TrainOutput(global_step=600, training_loss=3.789011262257894, metrics={'train_runtime': 16213.9067, 'train_samples_per_second': 1.173, 'train_steps_per_second': 0.037, 'total_flos': 1.5528270797733888e+17, 'train_loss': 3.789011262257894, 'epoch': 10.0})

## early LocalMinima

In [33]:
training_args_low_lr = TrainingArguments(
    output_dir="./qwen-qlora-it-2-v6-af",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    num_train_epochs=18,
    learning_rate=2e-5,   # much lower
    fp16=True,
    logging_steps=10,
    save_steps=200,
    report_to="none",
    lr_scheduler_type="cosine",   # or "linear", "polynomial", "cosine_with_restarts"
    warmup_steps=25    
)

trainer_low_lr = StableTrainer(
    model=model,
    args=training_args_low_lr,
    train_dataset=tokenized_train,
    data_collator=data_collator,
    json_penalty = 1
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Trainer.tokenizer is now deprecated. You should use `Trainer.processing_class = processing_class` instead.


In [34]:
trainer_low_lr.train(resume_from_checkpoint="/kaggle/working/qwen-qlora-it-2-v6/checkpoint-600")

Step,Training Loss
610,16.403700
620,17.569300
630,13.466200
640,9.414000
650,4.330800
660,2.819500
670,2.663300
680,2.574600
690,2.430500
700,2.458100


TrainOutput(global_step=1080, training_loss=1.2679100743046514, metrics={'train_runtime': 13009.7966, 'train_samples_per_second': 2.632, 'train_steps_per_second': 0.083, 'total_flos': 2.7950887435921e+17, 'train_loss': 1.2679100743046514, 'epoch': 18.0})

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_model_name = "Qwen/Qwen2.5-1.5B-Instruct"
adapter_checkpoint = "/kaggle/input/lora-best"   # pick your adapter

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Base model (old)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    # device_map="auto",
    # load_in_8bit=True,
    # torch_dtype="auto"
)

# Fine-tuned model (new)
finetuned_model = PeftModel.from_pretrained(base_model, adapter_checkpoint)


2025-11-29 20:47:35.834938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764449256.017947      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764449256.067433      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [36]:
import re
import json
from typing import Dict

def convert_colon_record_to_target(input_str: str,
                                   query: str = "",
                                   aet: int = 30,
                                   resolution_summary: str = "") -> str:

    # Regex finds keys at the start of lines like "KeyName:"
    key_pattern = re.compile(r'(?m)^(?P<key>[A-Za-z0-9_ ]+):')
    matches = list(key_pattern.finditer(input_str))

    parsed: Dict[str, str] = {}

    # Extract each key's value as substring from end of its colon to start of next key (or end)
    for i, m in enumerate(matches):
        key_raw = m.group('key').strip()
        start = m.end()  # position after the colon
        end = matches[i+1].start() if i+1 < len(matches) else len(input_str)
        value = input_str[start:end].strip()  # keep internal newlines
        # normalize multiple newlines/spaces at edges but preserve paragraphs
        value = re.sub(r'\r\n', '\n', value).strip()
        parsed[key_raw] = value

    # Map parsed keys (case-insensitive) to desired output keys
    key_map = {
        "symptoms": "Symptoms",
        "short_description": "Short_description",
        "long_description": "Long_description",
        "causes": "Causes",
        "resolution_note": "Resolution_note"
    }

    output = {
        "Query": query,
        "Symptoms": "",
        "Short_description": "",
        "Long_description": "",
        "Causes": "",
        "Resolution_note": ""
    }

    # Fill output from parsed values where possible
    for raw_k, v in parsed.items():
        lookup = raw_k.strip().lower().replace(' ', '_')
        target_key = key_map.get(lookup)
        if target_key:
            output[target_key] = v
        else:
            # If unknown key, add it with a readable name (capitalize, underscores)
            fallback_key = raw_k.strip().replace(' ', '_').capitalize()
            # avoid clobbering existing keys
            if fallback_key in output:
                # append with suffix if collision
                fallback_key = fallback_key + "_extra"
            output[fallback_key] = v

    # Produce pretty JSON and return inside json markdown block
    json_output = json.dumps(output, indent=2, ensure_ascii=False)
    return f"```json\n{json_output}\n```"


In [6]:



def generate(_model, system_prompt, user_query):
    user_query = f"{user_query} \n"
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ]
    print(messages)
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(_model.device)
    
    generated_ids = _model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    # return parser.invoke(convert_colon_record_to_target(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]))


In [5]:
prompts = [
    "HCL Domino server has been experiencing frequent crashes",
    "Excessive System Resources Usage by Riverbed SteelCentral NPM",
    "VLAN communication issue on Cisco Nexus switch"

]
import gc
import torch

# Delete any large objects (models, datasets) you no longer need
# del model
# del trainer
gc.collect()

# Clear PyTorch CUDA cache
torch.cuda.empty_cache()

# Optionally reset the CUDA memory allocator
torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()


In [89]:
results = []
for query in prompts:
    print("\n\n",("===="*25))
    for _type in list(["ko"]): #"causecode", "fishbone", "sip", "5_why", "ps"]):
        print(("----"*10),_type,("----"*10))
        reponse = generate(merged_model, system_prompts.get("ko"), query)
        print()
        print(("****"*5),"FINTUNED",("****"*5))
        print(reponse)
        results.append(reponse)
        



---------------------------------------- ko ----------------------------------------
[{'role': 'system', 'content': 'You are an expert IT assistant for Knowledge Article generation. Analyze the query and produce one complete KO object with detailed symptoms, descriptions, causes, and a 150+ word actionable Resolution_note. Output must strictly follow the JSON structure below exactly as written.\n```json\n{\n  "Query": "...",\n  "Symptoms": "...",\n  "Short_description": "...",\n  "Long_description": "...",\n  "Causes": "...",\n  "Resolution_note": ".."\n}\n```'}, {'role': 'user', 'content': 'HCL Domino server has been experiencing frequent crashes \n'}]

******************** FINTUNED ********************
```json
{
  "Symptoms": "The server is crashing frequently and unexpectedly, causing disruptions in service availability.",
  "Short_description": "Frequent crashes on HCL Domino server",
  "Long_description": "This article provides information on how to troubleshoot and resolve freq

In [94]:
# reponse = generate(base_model, system_prompts.get("ko"), "NCR Aloha Restaurant Enterprise Solution is not properly integrating with other software solutions")
# reponse

In [90]:
[parser.invoke(reponse) for respose in results]
# system_prompts.get("ko")

[KO(Symptoms='Devices on different VLANs cannot communicate with each other, even though they are connected to the same switch.', Short_description='This article explains how to troubleshoot VLAN communication issues on Cisco Nexus switches.', Long_description='When trying to communicate between devices on different VLANs on the same switch, it can be frustrating. This article will guide you through the steps to identify and resolve the issue.', Causes='The main cause of VLAN communication issues on Cisco Nexus switches is misconfiguration or incorrect tagging of VLANs. Other possible causes include network congestion, faulty cables, or hardware failures.', Resolution_note='To resolve the VLAN communication issue on a Cisco Nexus switch, follow these steps:<br><br>1. Verify that the VLANs are configured correctly on both ends of the network.<br>2. Check the tagging of the VLANs on the devices connected to the switch.<br>3. Use SNMP or command line access to diagnose the problem.<br>4. 

In [107]:
# import shutil

# # Zip the folder
shutil.rmtree("/kaggle/working/llama.cpp")


In [4]:
merged_model = finetuned_model.merge_and_unload()
merged_model.save_pretrained("merged_model")
tokenizer.save_pretrained("merged_model")

('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/chat_template.jinja',
 'merged_model/vocab.json',
 'merged_model/merges.txt',
 'merged_model/added_tokens.json',
 'merged_model/tokenizer.json')

In [47]:
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp


Cloning into 'llama.cpp'...
remote: Enumerating objects: 69587, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 69587 (delta 31), reused 18 (delta 18), pack-reused 69539 (from 2)
Receiving objects: 100% (69587/69587), 211.38 MiB | 39.06 MiB/s, done.
Resolving deltas: 100% (50439/50439), done.
python3: can't open file '/kaggle/working/convert.py': [Errno 2] No such file or directory


In [59]:
model.save_pretrained("base_model")
tokenizer.save_pretrained("base_model")

('base_model/tokenizer_config.json',
 'base_model/special_tokens_map.json',
 'base_model/chat_template.jinja',
 'base_model/vocab.json',
 'base_model/merges.txt',
 'base_model/added_tokens.json',
 'base_model/tokenizer.json')

In [76]:
!python3 convert_hf_to_gguf.py \
  /kaggle/working/llama.cpp/merged_model \
  --outfile /kaggle/working/my_model.Q8_0.gguf \
  --outtype q8_0

INFO:hf-to-gguf:Loading model: merged_model
INFO:hf-to-gguf:Model architecture: Qwen2ForCausalLM
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: indexing model part 'model-00002-of-00002.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00001-of-00002.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:blk.21.attn_norm.weight,   torch.bfloat16 --> F32, shape = {1536}
INFO:hf-to-gguf:blk.21.ffn_down.weight,    torch.int8 --> Q8_0, shape = {8960, 1536}
INFO:hf-to-gguf:blk.21.ffn_norm.weight,    torch.bfloat16 --> F32, shape = {1536}
INFO:hf-to-gguf:blk.22.attn_norm.weight,   torch.bfloat16 --> F32, shape = {1536}
INFO:hf-to-gguf:blk.22.ffn_down.weight,    torch.int8 --> Q8_0, shape = {8960, 1536}
INFO:hf-to-gguf:blk.22.ffn_gate.weight,    torch.int8 --> Q8_0, shape = {1536, 8960}

In [58]:
import shutil
import os

# Example: delete a folder named "llama.cpp"
folder_path = "/kaggle/working/merged_model"

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Deleted: {folder_path}")
else:
    print("Folder not found")


Deleted: /kaggle/working/merged_model


In [77]:
!zip -j /kaggle/outputs/my_model.zip /kaggle/working/my_model.Q8_0.gguf


zip I/O error: No such file or directory
zip error: Could not create output file (/kaggle/outputs/my_model.zip)


In [79]:
import os, shutil

# Make sure the outputs directory exists
os.makedirs("/kaggle/outputs", exist_ok=True)

# Now copy your file
shutil.copy("/kaggle/working/my_model.Q8_0.gguf", "/kaggle/outputs/my_model.Q8_0.gguf")


'/kaggle/outputs/my_model.Q8_0.gguf'

## hugging face

In [5]:

from huggingface_hub import login

# Paste your token here (keep it private!)
login("hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxx")


In [7]:
from huggingface_hub import upload_folder

# Path to your merged model
model_path = "/kaggle/working/merged_model"

# Your repo ID on Hugging Face (username/repo_name)
repo_id = "siddharth797/Qwen2.5-1.5b-full"

upload_folder(
    repo_id=repo_id,
    folder_path=model_path,
    commit_message="Upload merged fp16 model"
)


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/siddharth797/Qwen2.5-1.5b-full/commit/829951231d492ec90d22e008b4a54434e1833cee', commit_message='Upload merged fp16 model', commit_description='', oid='829951231d492ec90d22e008b4a54434e1833cee', pr_url=None, repo_url=RepoUrl('https://huggingface.co/siddharth797/Qwen2.5-1.5b-full', endpoint='https://huggingface.co', repo_type='model', repo_id='siddharth797/Qwen2.5-1.5b-full'), pr_revision=None, pr_num=None)

In [6]:
!huggingface-cli repo create Qwen2.5-1.5b-full

⚠️  Warning: 'huggingface-cli repo' is deprecated. Use 'hf repo' instead.
Successfully created siddharth797/Qwen2.5-1.5b-full on the Hub.
Your repo is now available at https://huggingface.co/siddharth797/Qwen2.5-1.5b-full


## ONNX

In [ ]:
!optimum-cli export onnx --model ./Qwen2.5-1.5b --task causal-lm ./onnx_qwen2

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# base_model_name = "siddharth797/Qwen2.5-1.5b"
base_model_name = "siddharth797/Qwen2.5-1.5b-full"

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    # load_in_4bit=True,
    # dtype="int4", 
    torch_dtype="auto",
)

model.save_pretrained("Qwen2.5-1.5b-ikon-full")
tokenizer.save_pretrained("Qwen2.5-1.5b-ikon-full")


2025-11-30 11:55:14.777797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764503714.979005      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764503715.039361      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

('Qwen2.5-1.5b-ikon-full/tokenizer_config.json',
 'Qwen2.5-1.5b-ikon-full/special_tokens_map.json',
 'Qwen2.5-1.5b-ikon-full/chat_template.jinja',
 'Qwen2.5-1.5b-ikon-full/vocab.json',
 'Qwen2.5-1.5b-ikon-full/merges.txt',
 'Qwen2.5-1.5b-ikon-full/added_tokens.json',
 'Qwen2.5-1.5b-ikon-full/tokenizer.json')

In [3]:
from optimum.onnxruntime import ORTModelForCausalLM
from transformers import AutoTokenizer

model_id = "siddharth797/Qwen2.5-1.5b-full"

# Export to ONNX with cache enabled
ort_model = ORTModelForCausalLM.from_pretrained(
    model_id,
    export=True,
    use_cache=True
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

ort_model.save_pretrained("./onnx_qwen2")
tokenizer.save_pretrained("./onnx_qwen2")


Multiple distributions found for package optimum. Picked distribution: optimum-onnx
2025-11-30 12:13:46.257884: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764504826.409575      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764504826.455615      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/cache_utils.py:568: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  or not self.key_cache[layer_idx].numel()  # the layer has no cache
/usr/local/lib/python3.11/dist-packages/transformers/masking_utils.py:187: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (padding_length := kv_length + kv_offset - attention_mask.shape[-1]) > 0:
/usr/local/lib/python3.11/dist-packages/transformers/masking_utils.py:215: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We ca

('./onnx_qwen2/tokenizer_config.json',
 './onnx_qwen2/special_tokens_map.json',
 './onnx_qwen2/chat_template.jinja',
 './onnx_qwen2/vocab.json',
 './onnx_qwen2/merges.txt',
 './onnx_qwen2/added_tokens.json',
 './onnx_qwen2/tokenizer.json')

In [10]:
for query in prompts:
    print("\n\n",("===="*25))
    for _type in ["ko"]:#, "causecode", "fishbone", "sip", "5_why", "ps"]:
        print(("----"*10),_type,("----"*10))
        reponse = generate(ort_model, system_prompts.get("ko"), query)
        print()
        print(("****"*5),"FINTUNED",("****"*5))
        print(reponse)
        



---------------------------------------- ko ----------------------------------------
[{'role': 'system', 'content': 'You are an expert IT assistant for Knowledge Article generation. Analyze the query and produce one complete KO object with detailed symptoms, descriptions, causes, and a 150+ word actionable Resolution_note. Output must strictly follow the JSON structure below exactly as written.\n```json\n{\n  "Query": "...",\n  "Symptoms": "...",\n  "Short_description": "...",\n  "Long_description": "...",\n  "Causes": "...",\n  "Resolution_note": ".."\n}\n```'}, {'role': 'user', 'content': 'HCL Domino server has been experiencing frequent crashes \n'}]

******************** FINTUNED ********************
```json
{
  "Symptoms": "The server is crashing frequently and unexpectedly, causing disruptions in service availability.",
  "Short_description": "Frequent crashes on HCL Domino server",
  "Long_description": "This article provides information on how to troubleshoot and resolve freq

In [11]:
from onnxruntime.quantization import quantize_dynamic, QuantType

# Paths
onnx_model_path = "/kaggle/working/onnx_qwen2/model.onnx"
quantized_model_path = "/kaggle/working/onnx_qwen2/model_int8.onnx"

# Quantize weights to INT8
quantize_dynamic(
    model_input=onnx_model_path,
    model_output=quantized_model_path,
    weight_type=QuantType.QInt8
)

print("✅ Quantized model saved at:", quantized_model_path)


✅ Quantized model saved at: /kaggle/working/onnx_qwen2/model_int8.onnx


In [14]:
import time
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForCausalLM

# Paths
base_dir = "/kaggle/working/onnx_qwen2"
int8_model_path = f"{base_dir}/model_int8.onnx"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_dir)

# Load baseline ONNX model
ort_model_fp32 = ORTModelForCausalLM.from_pretrained(base_dir, file_name="model.onnx")

# Load INT8 ONNX model
ort_model_int8 = ORTModelForCausalLM.from_pretrained(base_dir, file_name="model_int8.onnx")

# Test prompt
query = prompts[0]

def benchmark(model, name):
    inputs = tokenizer(query, return_tensors="pt")
    # Warmup
    _ = model.generate(**inputs, max_new_tokens=10, use_cache=False)

    # Benchmark
    start = time.time()
    outputs = generate(model, system_prompts.get("ko"), query)
    end = time.time()

    elapsed = end - start

    print(f"\n{name}:")
    print(outputs)
    print(f"Generated tokens in {elapsed:.2f} sec")

# Run both
benchmark(ort_model_fp32, "FP32 ONNX")
benchmark(ort_model_int8, "INT8 ONNX")


[{'role': 'system', 'content': 'You are an expert IT assistant for Knowledge Article generation. Analyze the query and produce one complete KO object with detailed symptoms, descriptions, causes, and a 150+ word actionable Resolution_note. Output must strictly follow the JSON structure below exactly as written.\n```json\n{\n  "Query": "...",\n  "Symptoms": "...",\n  "Short_description": "...",\n  "Long_description": "...",\n  "Causes": "...",\n  "Resolution_note": ".."\n}\n```'}, {'role': 'user', 'content': 'HCL Domino server has been experiencing frequent crashes \n'}]

FP32 ONNX:
```json
{
  "Symptoms": "The server is crashing frequently and unexpectedly, causing disruptions in service availability.",
  "Short_description": "Frequent crashes on HCL Domino server",
  "Long_description": "This article provides information on how to troubleshoot and resolve frequent crashes on an HCL Domino server. It covers the symptoms, causes, and step-by-step resolution process.",
  "Causes": "The u

In [2]:
!pip uninstall -y optimum optimum-onnxruntime onnxruntime-gpu onnxruntime
!pip install --upgrade optimum onnxruntime
!pip install "optimum[onnxruntime]"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 87.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip uninstall -y onnxruntime-gpu onnxruntime
!pip install onnxruntime[cpu]


Found existing installation: onnxruntime 1.23.2
Uninstalling onnxruntime-1.23.2:
  Successfully uninstalled onnxruntime-1.23.2
  Using cached onnxruntime-1.23.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.1 kB)
Using cached onnxruntime-1.23.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (17.4 MB)


In [4]:
import onnxruntime as ort
print(ort.get_device())  # should print "CPU"


CPU


In [10]:
# !git clone https://github.com/ggerganov/llama.cpp
# !cd /kaggle/working/llama.cpp
!pip install -r requirements.txt


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.6 MB/s

In [ ]:
# !git clone https://huggingface.co/siddharth797/Qwen2.5-1.5b-full

In [16]:
!python /kaggle/working/llama.cpp/convert_hf_to_gguf.py /kaggle/working/llama.cpp/Qwen2.5-1.5b-full --outtype bf16 --outfile Qwen3-8B-BF16.gguf

INFO:hf-to-gguf:Loading model: Qwen2.5-1.5b-full
INFO:hf-to-gguf:Model architecture: Qwen2ForCausalLM
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: indexing model part 'model-00002-of-00002.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00001-of-00002.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:blk.21.attn_norm.weight,   torch.float32 --> F32, shape = {1536}
INFO:hf-to-gguf:blk.21.ffn_down.weight,    torch.float32 --> BF16, shape = {8960, 1536}
INFO:hf-to-gguf:blk.21.ffn_norm.weight,    torch.float32 --> F32, shape = {1536}
INFO:hf-to-gguf:blk.22.attn_norm.weight,   torch.float32 --> F32, shape = {1536}
INFO:hf-to-gguf:blk.22.ffn_down.weight,    torch.float32 --> BF16, shape = {8960, 1536}
INFO:hf-to-gguf:blk.22.ffn_gate.weight,    torch.float32 --> BF16, shape = {1536, 8960}
INFO:hf-to-gguf:blk.22.ffn_up.weight,      torch.f

Cloning into 'Qwen2.5-1.5b-full'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 0), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (18/18), 1.72 MiB | 4.28 MiB/s, done.
Filtering content: 100% (3/3), 1.76 GiB | 41.34 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	model-00001-of-00002.safetensors

See: `git lfs help smudge` for more details.


In [17]:
!./build/bin/llama-quantize -h


/bin/bash: line 1: ./build/bin/llama-quantize: No such file or directory


In [18]:
!mkdir build
!cd build

In [26]:
!make -j


Makefile:6: *** Build system changed:
 The Makefile build has been replaced by CMake.

 For build instructions see:
 https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md

.  Stop.


In [31]:
LLAMA_DIR="/kaggle/working/llama.cpp"


# 3) Clean any previous Makefile/build artifacts that conflict with CMake
!cd "$LLAMA_DIR"
# remove old Makefile if present (safe if from previous build system)
!rm -f Makefile
# remove any old build dir to start fresh
!rm -rf build

# 4) Create build dir and run cmake from it
!mkdir -p build

In [46]:
# !rm -f Makefile
!rm -rf build
!mkdir build
!cd build

In [53]:
!sudo apt-get update -y
!sudo apt-get install -y build-essential cmake pkg-config libopenblas-dev libomp-dev

# go to llama.cpp folder
LLAMA_DIR="/kaggle/working/llama.cpp"

!echo "Switching to llama.cpp directory"
%cd $LLAMA_DIR

# remove stale cmake leftovers
!rm -f CMakeCache.txt
!rm -rf CMakeFiles
!rm -rf build

# create build folder
!mkdir build
%cd build

# run cmake with explicit generator & compilers
!cmake .. -G "Unix Makefiles" \
  -DCMAKE_BUILD_TYPE=Release \
  -DCMAKE_C_COMPILER=/usr/bin/gcc \
  -DCMAKE_CXX_COMPILER=/usr/bin/g++

# build with all CPU cores
!cmake --build . --config Release -j$(nproc)

# list the binaries
!echo "Built binaries in build/bin:"
!ls -la bin

# test the quantizer
!./bin/llama-quantize -h


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,157 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,498 kB]  
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports

In [71]:
# !cd /kaggle/working/llama.cpp

! ./bin/llama-quantize /kaggle/working/llama.cpp/Qwen3-8B-BF16.gguf Qwen3-8B-Q6_K.gguf Q6_K 8

main: build = 7225 (ed3208992)
main: built with gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/kaggle/working/llama.cpp/Qwen3-8B-BF16.gguf' to 'Qwen3-8B-Q6_K.gguf' as Q6_K using 8 threads
llama_model_loader: loaded meta data with 29 key-value pairs and 338 tensors from /kaggle/working/llama.cpp/Qwen3-8B-BF16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                     general.sampling.top_k i32              = 20
llama_model_loader: - kv   3:                     general.sampling.top_p f32              = 0.800000
llama_model_loader: - kv   4:                      general.sampling.temp f32              = 0.700000
llama_model_loader: - kv   5:   